In [2]:
import torch
import torch.nn.functional as F
import sys
import numpy as np

sys.path.append('/home/hxcai/cell_type_specific_CRE')
from MPRA_exp.datasets import SeqLabelDataset
from MPRA_exp.utils import *
from sei_framework.model.sei import Sei

In [2]:
def get_embedding(model, test_data_loader, device='cuda'):
    model = model.to(device)
    y_true = []
    y_pred = []
    embedding = []
    with torch.no_grad():
        model.eval()
        for (x, y) in tqdm(test_data_loader):
            x = x.to(device)
            x_rc = onehots_reverse_complement(x).to(device)
            
            pred_1, emb_1 = model.get_embedding(x)
            pred_2, emb_2 = model.get_embedding(x_rc)
            pred = (pred_1 + pred_2) / 2
            emb  = (emb_1 + emb_2) / 2
            # print(emb.cpu().detach().numpy())

            y_true.extend(y.cpu().detach().numpy())
            y_pred.extend(pred.cpu().detach().numpy())
            embedding.extend(emb.cpu().detach().numpy())

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    embedding = np.array(embedding)
    return y_true, y_pred, embedding

In [3]:
trained_model_path = '../sei_framework/model/sei.pth'
result_dir = './data'

sei_model = Sei().eval()
state_dict = torch.load(trained_model_path) 
new_state_dict = {k.replace('module.model.', ''): v for k, v in state_dict.items()}
sei_model.load_state_dict(new_state_dict)


<All keys matched successfully>

In [4]:
dataset = SeqLabelDataset(seq_exp_path='../data/GosaiMPRA/GosaiMPRA_len200.csv', input_column='seq', output_column='HepG2', seq_pad_len=4096)
test_data_loader = torch.utils.data.DataLoader(dataset, batch_size=512, shuffle=False)
y_true, y_pred, embedding = get_embedding(sei_model, test_data_loader)

In [4]:
np.save(f'{result_dir}/sei_joint_pred.npy', y_pred)
np.save(f'{result_dir}/sei_joint_embedding.npy', embedding)

In [ ]:
a = np.load('/home/hxcai/cell_type_specific_CRE/data/predicted/valid_labels.npy')